In [1]:

!pip install -q peft==0.8.2

In [4]:
%pip install accelerate peft bitsandbytes transformers trl

Note: you may need to restart the kernel to use updated packages.


In [5]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
NUM_VIRTUAL_TOKENS = 32
NUM_EPOCHS = 15



In [6]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline
   
)

2024-06-05 16:20:31.195809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 16:20:31.195914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 16:20:31.327590: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

In [8]:


model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    #quantization_config=quant_config,
    device_map={"": 0}, 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}
model.config.use_cache = False
model.config.pretraining_tp = 1


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
d = data[30]
import time
s=time.time()
print(d)
messages = [
    {"role": "user", "content": "only Answer the question based in  he best answers of this question "+str(d)},
    
]

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])
e=time.time()

print(e-s)

**Process dataset**

In [9]:
from datasets import load_dataset
dataset = load_dataset('json', data_files="/kaggle/input/prompt-data/prompting_data.json")
print (dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 31
    })
})


In [10]:
data_prompt = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
print (data_prompt)

train_sample_prompt = data_prompt["train"]
print (train_sample_prompt)


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 31
    })
})
Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 31
})


In [11]:
print(len(train_sample_prompt))

31


**Prompt Tuning**

In [12]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit


prompt_tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    prompt_tuning_init_text="answer the following question like the provided answer with no explanation",
    tokenizer_name_or_path=model_name,
)

In [17]:
peft_model_prompt = get_peft_model(model, prompt_tuning_config)
print(peft_model_prompt.print_trainable_parameters())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


trainable params: 98,304 || all params: 3,821,177,856 || trainable%: 0.002572609904708921
None


In [19]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.006, epochs=6):
    training_args = TrainingArguments(
        output_dir=path, 
        use_cpu=False, 
        auto_find_batch_size=True, 
        learning_rate= learning_rate, 
        num_train_epochs=epochs
    )
    return training_args

In [20]:
import os

working_dir = "./"

output_directory_prompt =  os.path.join(working_dir, "prompt_modelv4")


if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_directory_prompt):
    os.mkdir(output_directory_prompt)


In [21]:
training_args_prompt = create_training_arguments(output_directory_prompt, 0.0005, NUM_EPOCHS)


In [22]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=train_dataset, 
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) 
    )
    return trainer


In [ ]:
trainer_prompt = create_trainer(peft_model_prompt, training_args_prompt, train_sample_prompt)
trainer_prompt.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


**Save the model**

In [ ]:
trainer_prompt.model.save_pretrained(output_directory_prompt)


**Load the model**

In [ ]:
from peft import PeftModel
loaded_model = PeftModel.from_pretrained(model,"/kaggle/working/prompt_modelv4")
loaded_model = loaded_model.to("cuda")

In [ ]:
print(model)

In [ ]:
print(loaded_model)

In [ ]:

num_params_loaded_model = sum(p.numel() for p in loaded_model.parameters())
print("loaded model params :",num_params_loaded_model)
num_params = sum(p.numel() for p in model.parameters())
print("base model params :",num_params)
print(num_params_loaded_model - num_params)

In [ ]:
q = " question:Which NFL team won Super Bowl 50?"
ans="answer : Denver Broncos"
input_prompt = tokenizer("only answer "+q+ans, return_tensors="pt").to("cuda")


In [ ]:
foundational_outputs_prompt = get_outputs(loaded_model, input_prompt, max_new_tokens=100)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

In [ ]:
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.5,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

 **lora**

In [ ]:
q= "question:Which NFL team won Super Bowl 50?"
a = "answer : Denver Broncos"
prompt = f"Answer the question based on the answer provided: {q} {a}"
pipe = pipeline(task="question-answering", model="MistralForCausalLM", tokenizer=tokenizer, max_length=200,temperature = 0)
result = pipe(q,a)
print(result)

****